## LangChain的输出解释器

In [1]:
import os
from dotenv import load_dotenv

# 加载 .env 文件中的环境变量
load_dotenv(override=True)  # 使用 override=True 确保加载最新的 .env 数据

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=os.environ.get("OPENAPI_MODEL"),
    base_url=os.environ.get("OPENAPI_API_BASE"),
    api_key=os.environ.get("OPENAPI_API_KEY"),
    temperature=0,
)

question = "LangChain是什么？"

### 文本
+ 非标原生方式，直接获取模型的输出结果（不推荐），在Chain之间流转仍然需要封装为StrOutputParse
+ StrOutputParse （推荐），使用该输出解析便于在Chain之间流转

In [ ]:
response = llm.invoke("介绍下自己")
# 原生的返回结果
print(response)

content='你好呀！我是 **DeepSeek Chat**，由深度求索公司（DeepSeek）研发的智能 AI 助手。我可以帮你解答各种问题，包括学习、工作、编程、生活百科、创意写作等，还能阅读和解析上传的文件（如 PDF、Word、Excel 等）。  \n\n### ✨ **我的特点**  \n✅ **免费使用**：目前不收费，放心提问！  \n✅ **超长上下文**：支持 **128K** 上下文记忆，能处理超长文本。  \n✅ **文件阅读**：可以分析文档内容，帮你提取关键信息。  \n✅ **知识丰富**：我的知识截止到 **2024 年 7 月**，能提供较新的信息。  \n✅ **多语言支持**：可以用中文、英文等多种语言交流。  \n\n### 🚀 **我能帮你做什么？**  \n📖 **学习辅导**：解题思路、论文润色、知识点讲解  \n💼 **工作助手**：写邮件、做 PPT、数据分析、简历优化  \n💻 **编程支持**：代码调试、算法讲解、项目建议  \n🎨 **创意写作**：写故事、起名字、广告文案、诗歌创作  \n📂 **文件解析**：总结 PDF、Word、Excel 内容  \n\n如果你有任何问题，尽管问我吧！😊' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 278, 'prompt_tokens': 6, 'total_tokens': 284, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'deepseek-v3-250324', 'system_fingerprint': None, 'id': '021757035491663cebc2095a9c9da0efa65

In [12]:
# 使用 StrOutputParser 进行解析，可以大幅简化从LangChain中提取文本
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool


@tool
def get_weather(location: str) -> str:
    """根据location地名获取当地实时天气"""
    return f"{location}的天气是晴天，温度25度"


# 绑定工具
llm_with_tools = llm.bind_tools([get_weather])

chain = llm_with_tools | StrOutputParser()

response = chain.invoke("北京的天气怎么样")
print(response)


### Pydantic
+ PydanticOutputParser

In [25]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field, model_validator


# 定义一个名为Joke的数据模型
# 必须要包含的数据字段： setup（笑话的开头） 和 punchline（笑话的结尾）
class Joke(BaseModel):
    setup: str = Field(description="笑话中的铺垫问题，必须以？结尾")
    punchline: str = Field(
        description="笑话中回答铺垫问题的部分，通常是一种抖包袱方式回答铺垫问题，例如谐音、会错意等"
    )

    # 验证器，你可以根据自己的数据情况进行自定义
    # 注意mode=before意思是数据被转成pydantic模型的字段之前，对原始数据进行验证
    @model_validator(mode="before")
    @classmethod
    def question_ends_with_question_mark(cls, values: dict) -> dict:
        setup = values.get("setup")
        # 如果铺垫问题没有以问号结尾，则抛出异常
        if setup is not None and (not setup.endswith("？") and not setup.endswith("?")):
            raise ValueError("Badly formed question!")
        return values


# 实例化解析器、提示词模板
parser = PydanticOutputParser(pydantic_object=Joke)
print("PydanticOutputParser的格式指令要求为：")
print(parser.get_format_instructions())

# 注意，提示词模板中需要部分格式化解析器的格式要求format_instructions
prompt = PromptTemplate(
    template="回答用户的查询，\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
print("提示词模板为：")
print(prompt.invoke(""))


# 使用LCEL语法组合一个简单的链
prompt_and_model = prompt | llm
output = prompt_and_model.invoke({"query": "讲一个编程相关的笑话"})
print(output)
parser.invoke(output)

PydanticOutputParser的格式指令要求为：
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"setup": {"description": "笑话中的铺垫问题，必须以？结尾", "title": "Setup", "type": "string"}, "punchline": {"description": "笑话中回答铺垫问题的部分，通常是一种抖包袱方式回答铺垫问题，例如谐音、会错意等", "title": "Punchline", "type": "string"}}, "required": ["setup", "punchline"]}
```
提示词模板为：
text='回答用户的查询，\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}

Joke(setup='为什么程序员总是分不清万圣节和圣诞节？', punchline='因为 Oct 31 等于 Dec 25！')

### JSON
+ JsonOutputParser

In [33]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field, model_validator


# 定义一个名为Joke的数据模型
# 必须要包含的数据字段： setup（笑话的开头） 和 punchline（笑话的结尾）
class Joke(BaseModel):
    setup: str = Field(description="笑话中的铺垫问题，必须以？结尾")
    punchline: str = Field(
        description="笑话中回答铺垫问题的部分，通常是一种抖包袱方式回答铺垫问题，例如谐音、会错意等"
    )


# 实例化解析器、提示词模板
parser = JsonOutputParser(pydantic_object=Joke)
print("PydanticOutputParser的格式指令要求为：")
print(parser.get_format_instructions())

# 注意，提示词模板中需要部分格式化解析器的格式要求format_instructions
prompt = PromptTemplate(
    template="回答用户的查询，\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
print("提示词模板为：")
print(prompt.invoke(""))


# 使用LCEL语法组合一个简单的链
chain = prompt | llm | parser
output = chain.invoke({"query": "讲一个笑话"})
print(output)

PydanticOutputParser的格式指令要求为：
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"setup": {"description": "笑话中的铺垫问题，必须以？结尾", "title": "Setup", "type": "string"}, "punchline": {"description": "笑话中回答铺垫问题的部分，通常是一种抖包袱方式回答铺垫问题，例如谐音、会错意等", "title": "Punchline", "type": "string"}}, "required": ["setup", "punchline"]}
```
提示词模板为：
text='回答用户的查询，\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}

### XML
+ XMLOutputParser

In [37]:
! pip install defusedxml

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/07/6c/aa3f2f849e01cb6a001cd8554a88d4c77c5c1a31c95bdf1cf9301e6d9ef4/defusedxml-0.7.1-py2.py3-none-any.whl (25 kB)


In [39]:
from langchain_core.output_parsers import XMLOutputParser
from langchain_core.prompts import PromptTemplate

# 实例化解析器
# parser = XMLOutputParser()
# 方式二：指定标签
parser = XMLOutputParser(tags=["movies", "actor", "film", "name", "genre"])
print("XMLOutputParser的格式指令要求为：")
print(parser.get_format_instructions())


# 实例化提示词模板
prompt = PromptTemplate(
    template="{query}\n{format_instructions}",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
print("提示词模板为：")
print(prompt.invoke(""))

# 使用LCEL语法组合一个简单的链
chain = prompt | llm | parser
output = chain.invoke({"query": "Generate the shortend filmography for Tom Hanks."})
print(output)

XMLOutputParser的格式指令要求为：
The output should be formatted as a XML file.
1. Output should conform to the tags below.
2. If tags are not given, make them on your own.
3. Remember to always open and close all the tags.

As an example, for the tags ["foo", "bar", "baz"]:
1. String "<foo>
   <bar>
      <baz></baz>
   </bar>
</foo>" is a well-formatted instance of the schema.
2. String "<foo>
   <bar>
   </foo>" is a badly-formatted instance.
3. String "<foo>
   <tag>
   </tag>
</foo>" is a badly-formatted instance.

Here are the output tags:
```
['movies', 'actor', 'film', 'name', 'genre']
```
提示词模板为：
text='\nThe output should be formatted as a XML file.\n1. Output should conform to the tags below.\n2. If tags are not given, make them on your own.\n3. Remember to always open and close all the tags.\n\nAs an example, for the tags ["foo", "bar", "baz"]:\n1. String "<foo>\n   <bar>\n      <baz></baz>\n   </bar>\n</foo>" is a well-formatted instance of the schema.\n2. String "<foo>\n   <bar>\n 